In [ ]:
#============================ Loading required libraries ============================#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import seaborn
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
#============================ Step 1 Data Cleaning ============================#
# Load the data
file_path_framingham = '../2. datasets/Framingham_preprocessed_data.csv'
file_path_stroke = '../2. datasets/StrokeHealthcareData.csv'
file_path_clinical = '../2. datasets/HeartFailureClinicalRecords_data519.csv'
file_path_statlog = '../2. datasets/StatlogHeart_data145.csv'
file_path_cleveland = '../2. datasets/ClevelandData.csv'
file_path_hungary = '../2. datasets/HungaryData.csv'
file_path_swizerland = '../2. datasets/SwitzerlandData.csv'
file_path_valongbeach = '../2. datasets/VaLongBeachData.csv'

cleveland = pd.read_csv(file_path_cleveland)
hungary = pd.read_csv(file_path_hungary)
switzerland = pd.read_csv(file_path_swizerland)
valongbeach = pd.read_csv(file_path_valongbeach)
statlog = pd.read_csv(file_path_statlog)
clinical = pd.read_csv(file_path_clinical)
stroke = pd.read_csv(file_path_stroke)
framingham = pd.read_csv(file_path_framingham)

# Combine the datasets
datasets_1 = [cleveland, hungary, switzerland, valongbeach]
# datasets_2 = [statlog, clinical, stroke, framingham]
datasets= [
    df.drop(columns=['id', 'dataset'], errors='ignore') for df in datasets_1
]


Combined dataset saved to ../3. clean data/dataset_1.csv


In [21]:


# Replace 'cp' with 'chest pain' in the 'Symptom' column
datasets_1 = datasets_1.rename(columns={'cp': 'chest-pain'})
datasets_1 = datasets_1.rename(columns={'fbs': 'fasting-blood-sugar'})
datasets_1 = datasets_1.rename(columns={'trestbps': 'rest-bps'})
datasets_1 = datasets_1.rename(columns={'chol': 'serum-chol'})
datasets_1 = datasets_1.rename(columns={'thalch': 'max-heart-rate'})
datasets_1 = datasets_1.rename(columns={'ca': 'major-vessels'})
datasets_1 = datasets_1.rename(columns={'resting-ecg': 'restecg'})
datasets_1 = datasets_1.rename(columns={'exang': 'exercise-angina'})
datasets_1 = datasets_1.rename(columns={'num': 'heart-disease'})
datasets_1 = datasets_1.rename(columns={'oldpeak': 'st-depression'})



datasets_2['sex'] = datasets_2['sex'].replace({'male': 1, 'female': 0})
datasets_2['chest-pain'] = datasets_2['chest-pain'].replace({'typical angina': 0, 'atypical angina': 1, 'non-anginal pain': 2, 'atypical angina': 3})
datasets_2['fasting-blood-sugar'] = datasets_2['fasting-blood-sugar'].replace({'TRUE': 1, 'FALSE': 0})
datasets_2['restecg'] = datasets_2['restecg'].replace({'normal': 0, 'st-t abnormality': 1, 'lv hypertrophy': 2})
datasets_2['exercise-angina'] = datasets_2['exercise-angina'].replace({'TRUE': 1, 'FALSE': 0})
datasets_2['slope'] = datasets_2['slope'].replace({'male': 1, 'female': 0})
datasets_2['thal'] = datasets_2['thal'].replace({'male': 1, 'female': 0})
# Check the number of unique values and list the unique values for each column
# for column in datasets_1.columns:
#     unique_values = datasets_1[column].unique()  # Get unique values
#     num_unique_values = datasets_1[column].nunique()  # Count unique values
    
#     print(f"Column: {column}")
#     print(f"Number of Unique Values: {num_unique_values}")
#     print(f"Unique Values: {unique_values}\n")


statlog = statlog.rename(columns={'rest-bp': 'rest-bps'})
statlog = statlog.rename(columns={'electrocardiographic': 'restecg'})
statlog = statlog.rename(columns={'angina': 'exercise-angina'})
statlog = statlog.rename(columns={'oldpeak': 'st-depression'})

datasets = [datasets_1, statlog]
datasets_2 = pd.concat(datasets, ignore_index=True)

output_path = "../3. clean data/dataset_2.csv"
datasets_2.to_csv(output_path, index=False)
print(f"Combined dataset saved to {output_path}")
for column in datasets_2.columns:
    unique_values = datasets_2[column].unique()  # Get unique values
    num_unique_values = datasets_2[column].nunique()  # Count unique values
    
    print(f"Column: {column}")
    print(f"Number of Unique Values: {num_unique_values}")
    print(f"Unique Values: {unique_values}\n")



# Encode categorical features
# Handle missing values
# Split the data into training and testing sets

#============================ Step 2 Generate a Common Feature Space ============================#

Combined dataset saved to ../3. clean data/dataset_2.csv
Column: age
Number of Unique Values: 50
Unique Values: [63. 67. 37. 41. 56. 62. 57. 53. 44. 52. 48. 54. 49. 64. 58. 60. 50. 66.
 43. 40. 69. 59. 42. 55. 61. 65. 71. 51. 46. 45. 39. 68. 47. 34. 35. 29.
 70. 77. 38. 74. 76. 28. 30. 31. 32. 33. 36. 72. 73. 75.]

Column: sex
Number of Unique Values: 4
Unique Values: ['Male' 'Female' 1.0 0.0]

Column: chest-pain
Number of Unique Values: 8
Unique Values: ['typical angina' 'asymptomatic' 'non-anginal' 'atypical angina' 4.0 3.0
 2.0 1.0]

Column: rest-bps
Number of Unique Values: 61
Unique Values: [145. 160. 120. 130. 140. 172. 150. 110. 132. 117. 135. 112. 105. 124.
 125. 142. 128. 170. 155. 104. 180. 138. 108. 134. 122. 115. 118. 100.
 200.  94. 165. 102. 152. 101. 126. 174. 148. 178. 158. 192. 129. 144.
 123. 136. 146. 106. 156. 154. 114. 164.  98. 190.  nan 113.  92.  95.
  80. 185. 116.   0.  96. 127.]

Column: serum-chol
Number of Unique Values: 217
Unique Values: [233. 286. 229. 2

In [18]:
# Calculate total missing values per column
for column in datasets_1.columns:
    missing_values = datasets_1[column].isnull().sum()
    total_values = len(datasets_1[column])
    missing_percentage = (datasets_1[column].isnull().sum() / len(datasets_1[column])) * 100
    print(f"Column: {column}")
    print(f"Total Values {total_values}")
    print(f"Missing Values {missing_values}")
    print(f"Percentage {missing_percentage}\n")

Column: age
Total Values 920
Missing Values 0
Percentage 0.0

Column: sex
Total Values 920
Missing Values 0
Percentage 0.0

Column: chest-pain
Total Values 920
Missing Values 0
Percentage 0.0

Column: rest-bps
Total Values 920
Missing Values 59
Percentage 6.41304347826087

Column: serum-chol
Total Values 920
Missing Values 30
Percentage 3.260869565217391

Column: fasting-blood-sugar
Total Values 920
Missing Values 90
Percentage 9.782608695652174

Column: resting-ecg
Total Values 920
Missing Values 2
Percentage 0.21739130434782608

Column: max-heart-rate
Total Values 920
Missing Values 55
Percentage 5.978260869565218

Column: exercise-angina
Total Values 920
Missing Values 55
Percentage 5.978260869565218

Column: oldpeak
Total Values 920
Missing Values 62
Percentage 6.739130434782608

Column: slope
Total Values 920
Missing Values 309
Percentage 33.58695652173913

Column: major-vessels
Total Values 920
Missing Values 611
Percentage 66.41304347826087

Column: thal
Total Values 920
Missing